In [1]:
import os
import json
from dotenv import load_dotenv
from openai import AsyncOpenAI
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langchain.messages import HumanMessage
from pathlib import Path

from dataclasses import dataclass
from typing import Dict, List, Optional, Literal

# Read the environment variables
load_dotenv()

# Initialize OpenAI client
openai_client = AsyncOpenAI(api_key=os.environ["OPENAI_API_KEY"])

In [2]:
# Set path
CACHE_FILE = "./file_cache_openai.json"
paper_pdf_path = "../.example/paper.pdf"
solution_pdf_path = "../.example/solution.pdf"

def load_cache():
    """Read the cache in JSON format"""
    if os.path.exists(CACHE_FILE):
        with open(CACHE_FILE, "r", encoding="utf-8") as f:
            return json.load(f)
    return {}

def save_cache(cache: dict):
    """Save the cache in JSON format"""
    with open(CACHE_FILE, "w", encoding="utf-8") as f:
        json.dump(cache, f, indent=2, ensure_ascii=False)

async def upload_if_needed(path: str, cache_key: str):
    """
    Get the file ID if it has already been uploaded.
    If not, upload the file to OpenAI and return the file ID.
    
    Note: OpenAI uses file IDs, not URIs
    """
    # Load the cache
    cache = load_cache()

    # If the file ID is already cached, return it.
    if cache_key in cache:
        file_id = cache[cache_key]
        print(f"✅ 已缓存: {cache_key} → {file_id}")
        
        # Verify file still exists
        try:
            await openai_client.files.retrieve(file_id)
            return file_id
        except Exception as e:
            print(f"⚠️ 缓存的文件已失效，重新上传...")
            cache.pop(cache_key, None)

    # Upload file to OpenAI
    print(f"⏫ 正在上传到 OpenAI: {path} ...")
    with open(path, 'rb') as f:
        uploaded = await openai_client.files.create(
            file=f,
            purpose="assistants"  # OpenAI 需要指定用途
        )
    
    file_id = uploaded.id
    cache[cache_key] = file_id
    save_cache(cache)
    print(f"✅ 上传完成: {cache_key} → {file_id}")
    return file_id

async def list_all_uploaded_files():
    """列出所有已上传的文件"""
    files = await openai_client.files.list()
    
    print("\n📁 OpenAI 已上传文件列表:")
    for f in files.data:
        print(f"""
    ID:       {f.id}
    Filename: {f.filename}
    Purpose:  {f.purpose}
    Size:     {f.bytes / 1024:.1f} KB
    Created:  {f.created_at}
        """)

async def delete_all_files():
    """删除所有已上传的文件"""
    files = await openai_client.files.list()
    
    for f in files.data:
        await openai_client.files.delete(f.id)
        print(f"🗑️ 已删除: {f.id} ({f.filename})")
    
    print(f"✅ 已删除 {len(files.data)} 个文件")


# 上传 PDF 文件
paper_file_id = await upload_if_needed(paper_pdf_path, "paper_pdf_id")
solution_file_id = await upload_if_needed(solution_pdf_path, "solution_pdf_id")

print(f"\n📄 Paper File ID: {paper_file_id}")
print(f"📄 Solution File ID: {solution_file_id}")

✅ 已缓存: paper_pdf_id → file-1MVpD6pbVE2henLJ4uiPvf
✅ 已缓存: solution_pdf_id → file-ChAcwXYkWKiuYT54Pdm4LQ

📄 Paper File ID: file-1MVpD6pbVE2henLJ4uiPvf
📄 Solution File ID: file-ChAcwXYkWKiuYT54Pdm4LQ


In [3]:
# 列出所有已上传的文件
await list_all_uploaded_files()



📁 OpenAI 已上传文件列表:

    ID:       file-ChAcwXYkWKiuYT54Pdm4LQ
    Filename: solution.pdf
    Purpose:  assistants
    Size:     9230.1 KB
    Created:  1762882475
        

    ID:       file-1MVpD6pbVE2henLJ4uiPvf
    Filename: paper.pdf
    Purpose:  assistants
    Size:     518.9 KB
    Created:  1762882457
        

    ID:       file-4VtAqpn7qJfL6QZHKTs4jp
    Filename: first5_solution.pdf
    Purpose:  assistants
    Size:     835.5 KB
    Created:  1762433905
        

    ID:       file-4HKYQitwgmU74U1Q1puaVN
    Filename: solution.pdf
    Purpose:  assistants
    Size:     9230.1 KB
    Created:  1762430651
        

    ID:       file-F4sS11gdBtzsCJKY4UHknK
    Filename: paper.pdf
    Purpose:  assistants
    Size:     518.9 KB
    Created:  1762430617
        


In [4]:
# # System_head = """
# # You are an Australian high school teacher.
# # You are asked to perform tasks related to the processing and analysis of exam papers to assist with student learning.
# # """

# @dataclass
# class FlowContext:
#     """
#     Customized data class for flow
#     """
#     step: Literal["classifier", "question_lister"]
#     exam_id: str
#     exam_type: Optional[Literal["type1", "type2"]] = None


In [5]:
# class PromptRegistry:
#     def __init__(self):
#         """
#         Manage all prompts for the flow.
#         self._sections: Dict[str, List[str]] is a dictionary that stores the prompt sections for different steps.
#         """
#         self._sections: Dict[str, List[str]] = {}
    
#     def register(self, step: str, *sections: str) -> None:
#         """Register a step's prompt sections"""
#         self.sections.setdefault(step, []).extend([s.strip() for s in sections if s])
    
#     def build(self, step: str, **vars) -> str:
#         """Build the full prompt for a step, supporting variable interpolation"""
#         parts = self.sections.get(step, [])
#         if not parts:
#             raise ValueError(f"No prompt registered for step: {step}")
        
#         # Simple variable interpolation (support str.format)
#         blob = "\n\n".join(
#             s.format(**vars) if "{" in s else s
#             for s in parts
#         )
#         return blob

#     def _register_classifier(self) -> None:
#         """Register the classifier prompt sections"""
#         self.register(
#             "classify",
#             """Analyze the provided pages of this exam and determine its type.""",
            
#             """**Type1** (Separate Answer Booklet):
#                 - Explicitly states "Use a SEPARATE writing booklet" or similar
#                 - No blank lines or answer spaces under questions
#                 - Questions are densely packed
#                 - Example: "10(a)", "10(b)", "10(c)" are independent questions""",
            
#             """**Type2** (Answer on Paper):
#                 - Has blank lines or answer spaces under questions, including:
#                 * Underscores (______)
#                 * Dotted lines (..................)
#                 * Multiple blank lines for writing answers
#                 - Clear answer spaces between questions
#                 - Questions have more spacing
#                 - Example: "Question 21" is one complete question with sub-parts (a), (b), (c)""",
                            
#             """**Analysis Guidelines**:
#                 1. Look for explicit instructions about where to write answers
#                 2. Check for blank answer spaces or lines
#                 3. Observe question density and spacing
#                 4. Note the question numbering pattern""",
                            
#             """Return JSON with:
#                 {{
#                     "exam_type": "type1" or "type2",
#                     "reasoning": "Detailed explanation of classification decision",
#                     "confidence": 0.0-1.0 (optional)
#                 }}""",

#             """**Important**: Base your decision on multiple indicators, not just one feature."""
#         )

In [6]:
from __future__ import annotations
from dataclasses import dataclass, field
from typing import Dict, List, Optional, Literal
from pathlib import Path

# ============= 业务上下文（传递给 runtime.context） =============

@dataclass
class FlowContext:
    """
    V4 File-Based Workflow 的运行时上下文
    用于在不同 Agent 步骤间传递状态和配置信息
    """
    
    # ========== 步骤控制 ==========
    step: Literal[
        "classify",           # Step 0: 分类试卷类型
        "lister",             # Step 1: 列出所有题目
        "annotate_paper",     # Step 1.5: 标注题目在 paper 中的页码
        "annotate_solution",  # Step 1.5: 标注题目在 solution 中的页码
        "question_latex",     # Step 2: 生成题目 LaTeX
        "answer_latex",       # Step 3: 生成答案 LaTeX
        "bbox_correction",    # Step 4: 图片 bbox 修正
        "labelling"           # Step 5: 题目标注（topic/subtopic/type/difficulty）
    ]
    
    # ========== 试卷元数据 ==========
    exam_id: str                              # 试卷唯一标识
    exam_type: Optional[Literal["type1", "type2"]] = None  # 试卷类型（classify 后填充）
    
    # ========== 文件路径 ==========
    paper_pdf_path: Optional[str] = None      # 原始 paper PDF 路径
    solution_pdf_path: Optional[str] = None   # 原始 solution PDF 路径
    paper_file_id: Optional[str] = None       # OpenAI 上传的 paper 文件 ID
    solution_file_id: Optional[str] = None    # OpenAI 上传的 solution 文件 ID
    output_dir: Optional[Path] = None         # 输出目录
    
    # ========== 题目信息（lister 后填充） ==========
    total_questions: int = 0                  # 题目总数
    current_question_index: Optional[int] = None    # 当前处理的题目索引（1-based）
    current_question_label: Optional[str] = None    # 当前处理的题目标签（如 "10(a)"）
    paper_pages: Optional[List[int]] = None         # 当前题目在 paper 中的页码（0-based）
    solution_pages: Optional[List[int]] = None      # 当前题目在 solution 中的页码（0-based）
    
    # ========== 标注相关（labelling 阶段使用） ==========
    subject_id: Optional[int] = None          # Subject ID（用于获取 subtopics）
    grade_id: Optional[int] = None            # Grade ID（用于获取 subtopics）
    existing_mark: Optional[int] = None       # 已提取的分数（传递给 labelling）
    
    # ========== 中间件控制参数 ==========
    token_budget: int = 16000                 # 中间件用于裁剪长 prompt 的上限
    max_tokens: int = 8000                    # API 请求的 max_tokens 限制
    temperature: float = 0.0                  # 生成温度
    enable_retry: bool = True                 # 是否启用重试机制
    retry_count: int = 0                      # 当前重试次数
    max_retries: int = 2                      # 最大重试次数
    
    # ========== 工具控制 ==========
    plan: Literal["basic", "pro", "premium"] = "pro"  # 用于裁剪工具的计划等级
    available_tools: List[str] = field(default_factory=list)  # 当前步骤可用的工具列表
    
    # ========== 图片处理相关 ==========
    enable_bbox_correction: bool = True       # 是否启用 bbox 修正
    bbox_correction_max_iterations: int = 4   # Bbox 修正最大迭代次数
    render_quality: Literal["low", "medium", "high"] = "medium"  # PDF 渲染质量
    
    # ========== 输出格式控制 ==========
    response_format: Literal["json_object", "text"] = "json_object"  # API 响应格式
    enable_output_validation: bool = True     # 是否启用输出校验
    
    # ========== 性能优化 ==========
    enable_concurrent: bool = True            # 是否启用并发处理
    concurrent_batch_size: int = 5            # 并发批次大小
    
    # ========== 调试和日志 ==========
    debug_mode: bool = False                  # 调试模式
    save_intermediate_results: bool = True    # 是否保存中间结果
    log_level: Literal["DEBUG", "INFO", "WARNING", "ERROR"] = "INFO"
    
    # ========== 特定步骤的额外配置 ==========
    extra_config: Dict = field(default_factory=dict)  # 存储步骤特定的额外配置
    
    # ========== 辅助方法 ==========
    
    def is_type1(self) -> bool:
        """判断是否为 type1 试卷"""
        return self.exam_type == "type1"
    
    def is_type2(self) -> bool:
        """判断是否为 type2 试卷"""
        return self.exam_type == "type2"
    
    def get_question_context(self) -> str:
        """获取当前题目的上下文描述"""
        if self.current_question_label:
            return f"Question {self.current_question_label} (index: {self.current_question_index})"
        return "No question context"
    
    def get_step_config(self) -> Dict:
        """
        根据当前步骤返回对应的配置
        可用于中间件动态调整参数
        """
        config = {
            "step": self.step,
            "token_budget": self.token_budget,
            "max_tokens": self.max_tokens,
            "temperature": self.temperature,
            "response_format": self.response_format,
        }
        
        # 步骤特定配置
        if self.step == "classify":
            config.update({
                "max_tokens": 2000,
                "enable_tools": False,  # 分类不需要工具
            })
        elif self.step == "lister":
            config.update({
                "max_tokens": 16000,  # Question list 可能较长
                "enable_tools": True,
                "available_tools": ["file_search"],
                "retry_on_validation_failure": True,
            })
        elif self.step in ["question_latex", "answer_latex"]:
            config.update({
                "max_tokens": 8000,
                "enable_tools": False,
                "enable_image_extraction": True,
            })
        elif self.step == "bbox_correction":
            config.update({
                "max_tokens": 1000,
                "model": "gpt-5",  # 使用更强的模型
                "temperature": 0.0,
            })
        elif self.step == "labelling":
            config.update({
                "max_tokens": 3000,
                "enable_tools": False,
                "require_subtopic_list": True,
            })
        
        # 合并 extra_config
        if self.extra_config:
            config.update(self.extra_config)
        
        return config
    
    def clone_for_step(
        self, 
        step: str, 
        question_index: Optional[int] = None,
        question_label: Optional[str] = None,
        **kwargs
    ) -> "FlowContext":
        """
        克隆上下文并更新为新步骤
        用于在 workflow 中切换步骤时保持状态
        """
        from copy import deepcopy
        new_context = deepcopy(self)
        new_context.step = step
        
        if question_index is not None:
            new_context.current_question_index = question_index
        if question_label is not None:
            new_context.current_question_label = question_label
        
        # 更新其他传入的参数
        for key, value in kwargs.items():
            if hasattr(new_context, key):
                setattr(new_context, key, value)
        
        return new_context
    
    def to_dict(self) -> Dict:
        """转换为字典（用于日志或序列化）"""
        from dataclasses import asdict
        return asdict(self)

In [7]:
"""Prompt Registry for V4 File-Based Workflow"""

from typing import Dict, List, Optional
from pathlib import Path


class PromptRegistry:
    """集中管理不同步骤的大段 prompt 片段，可按 exam_type 等变量渲染。"""
    
    def __init__(self) -> None:
        self._sections: Dict[str, List[str]] = {}
        self._initialized = False
    
    def register(self, step: str, *sections: str) -> None:
        """注册一个步骤的 prompt 片段"""
        self._sections.setdefault(step, []).extend([s.strip() for s in sections if s])
    
    def build(self, step: str, **vars) -> str:
        """构建指定步骤的完整 prompt，支持变量插值"""
        parts = self._sections.get(step, [])
        if not parts:
            raise ValueError(f"No prompt registered for step: {step}")
        
        # 简单变量插值（支持 str.format）
        blob = "\n\n".join(
            s.format(**vars) if "{" in s else s
            for s in parts
        )
        return blob
    
    def register_all(self) -> None:
        """注册所有步骤的 prompts（从现有 agents 提取）"""
        if self._initialized:
            return
        
        self._register_classifier()
        self._register_lister()
        self._register_page_annotator()
        self._register_question_latex()
        self._register_answer_latex()
        self._register_bbox_corrector()
        self._register_labelling()
        
        self._initialized = True
    
    # ============= Step 0: Classifier =============
    
    def _register_classifier(self) -> None:
        """注册分类器 prompt"""
        self.register(
            "classify",
            """Analyze the provided pages of this exam and determine its type.""",
            
            """**Type1** (Separate Answer Booklet):
- Explicitly states "Use a SEPARATE writing booklet" or similar
- No blank lines or answer spaces under questions
- Questions are densely packed
- Example: "10(a)", "10(b)", "10(c)" are independent questions""",
            
            """**Type2** (Answer on Paper):
- Has blank lines or answer spaces under questions, including:
  * Underscores (______)
  * Dotted lines (..................)
  * Multiple blank lines for writing answers
- Clear answer spaces between questions
- Questions have more spacing
- Example: "Question 21" is one complete question with sub-parts (a), (b), (c)""",
            
            """**Analysis Guidelines**:
1. Look for explicit instructions about where to write answers
2. Check for blank answer spaces or lines
3. Observe question density and spacing
4. Note the question numbering pattern""",
            
            """Return JSON with:
{{
    "exam_type": "type1" or "type2",
    "reasoning": "Detailed explanation of classification decision",
    "confidence": 0.0-1.0 (optional)
}}

**Important**: Base your decision on multiple indicators, not just one feature."""
        )
    
    # ============= Step 1: Lister =============
    
    def _register_lister(self) -> None:
        """注册列题器 prompt"""
        
        # Type1 规则
        type1_rules = """【Type1 Rules】(Separate Answer Booklet):
- Question 10 is a **section title**, not an independent question
- 10(a), 10(b), 10(c) are **independent questions** (minimum splitting unit)
- 10(c)(i), 10(c)(ii) are **sub-parts** of 10(c), NOT separate questions
- Recognition pattern: ^\\d+\\([a-z]\\)$ indicates start of independent question

Example:
  10          ← Section title, NOT a question
  10(a)       ← Question 1: "10(a)"
  10(b)       ← Question 2: "10(b)"
  10(c)       ← Question 3: "10(c)"
    (i)       ← Sub-part of 10(c), NOT separate
    (ii)      ← Sub-part of 10(c), NOT separate
  11(a)       ← Question 4: "11(a)" """
        
        # Type2 规则
        type2_rules = """【Type2 Rules】(Answer on Paper):
- Each "Question N" (where N is a number) is **one complete question** (minimum splitting unit)
- Sub-parts like (a), (b), (c) are NOT separate questions
- Recognition pattern: ^Question \\d+$ indicates start of question
- **IMPORTANT**: Include ALL questions from Question 1 onwards (including short/multiple-choice questions at the beginning)

Example:
  Question 1     ← Question 1: "Question 1" (may be a short/multiple-choice question)
  Question 2     ← Question 2: "Question 2"
  ...
  Question 11    ← Question 11: "Question 11" (may have sub-parts below)
    (a)          ← Sub-part of Question 11, NOT separate
    (b)          ← Sub-part of Question 11, NOT separate
  Question 12    ← Question 12: "Question 12"
    (a)          ← Sub-part of Question 12, NOT separate"""
        
        self.register(
            "lister",
            """You are a Question Lister Agent. Your task is to scan the entire paper PDF and create a **complete, accurate list** of all questions.""",
            
            """=== Exam Type ===
{exam_type}""",
            
            """=== Question Splitting Rules ===
{cutting_rules}
{emphasis}""",
            
            """=== Your Task ===
1. Use the file_search tool to systematically scan the entire paper PDF
2. Identify ALL questions in the document (from Question 1 to the last question)
3. For each question, record:
   - question_index: Sequential number starting from 1 (1, 2, 3, ...)
   - question_label: **Exact label** as it appears in the paper (e.g., "10(a)", "Question 1", "Question 21")""",
            
            """=== Search Strategy ===
- **Start from Question 1** (or the first question in the document)
- Search for ALL question patterns systematically (don't skip short/multiple-choice questions at the beginning)
- Scan through the entire document from beginning to end
- Verify you've reached the last question
- Double-check the count and ensure Question 1 is included""",
            
            """=== Critical Rules ===
✅ DO:
- Follow the splitting rules **strictly**
- **Start from Question 1** - don't skip early questions
- Include ALL questions: short questions, multiple-choice questions, AND longer questions with sub-parts
- Preserve exact question labels (including parentheses, capitalization)
- Number questions sequentially (1, 2, 3, ...)
- Include ALL questions, no matter how short

❌ DON'T:
- Skip the first few questions (e.g., Question 1-10)
- Split sub-parts into separate questions
- Guess or skip questions
- Change the question labels
- Include section titles as questions (for type1)""",
            
            """=== Output Format ===
**IMPORTANT: Return your response as a JSON object** with the following structure:
{{
    "exam_type": "{exam_type}",
    "total_questions": <count>,
    "questions": [
        {{"question_index": 1, "question_label": "..."}},
        {{"question_index": 2, "question_label": "..."}},
        ...
    ]
}}""",
            
            """=== Quality Check ===
Before returning, verify:
1. total_questions == len(questions)
2. question_index are sequential (1, 2, 3, ...)
3. No duplicate question_labels
4. All question_labels follow the format rules

Begin scanning now using the file_search tool. Be thorough and accurate!"""
        )
        
        # 保存规则模板以便动态渲染
        self._sections["_lister_type1_rules"] = [type1_rules]
        self._sections["_lister_type2_rules"] = [type2_rules]
    
    def _get_lister_emphasis(self, exam_type: str, emphasize: bool) -> str:
        """生成 lister 的强调文本（重试时使用）"""
        if not emphasize:
            return ""
        
        if exam_type == "type1":
            return """

⚠️ **CRITICAL REMINDER for Type1**:
- You MUST split questions to the (a), (b), (c) level
- DO NOT list "Question 10" or "Question 11" as single questions
- EVERY question label should contain (a), (b), (c), etc.
- Example: If you see "10(a)", "10(b)", "10(c)", list them as THREE separate questions
- Pattern to follow: "10(a)", "10(b)", "10(c)", "11(a)", "11(b)", etc.
- This is a Type1 exam with SEPARATE answer booklet - questions are split into sub-parts!"""
        else:  # type2
            return """

⚠️ **CRITICAL REMINDER for Type2**:
- You MUST NOT split (a), (b), (c) into separate questions
- DO NOT list "10(a)", "10(b)" as separate questions
- List ONLY "Question N" format (e.g., "Question 1", "Question 2")
- If a question has sub-parts (a), (b), (c), they are ALL part of ONE question
- Example: "Question 11" with (a), (b), (c) below = ONE question labeled "Question 11"
- This is a Type2 exam - answers are written ON the paper, sub-parts are NOT separate!"""
    
    def build_lister_prompt(self, exam_type: str, emphasize: bool = False) -> str:
        """构建 lister prompt（特殊处理 exam_type 和 emphasis）"""
        # 获取切题规则
        if exam_type == "type1":
            cutting_rules = self._sections.get("_lister_type1_rules", [""])[0]
        else:
            cutting_rules = self._sections.get("_lister_type2_rules", [""])[0]
        
        # 获取强调文本
        emphasis = self._get_lister_emphasis(exam_type, emphasize)
        
        return self.build(
            "lister",
            exam_type=exam_type,
            cutting_rules=cutting_rules,
            emphasis=emphasis
        )
    
    # ============= Step 1.5: Page Annotator =============
    
    def _register_page_annotator(self) -> None:
        """注册页码标注 prompt"""
        
        # Paper pages annotator
        self.register(
            "annotate_paper",
            """You are analyzing a paper PDF with page markers.

The PDF has visible PAGE_INDEX_N markers (0-based) at the top-right of each page.""",
            
            """Here are the questions in this paper:
{questions_list}""",
            
            """For EACH question, identify ALL pages where it appears (may span multiple pages).""",
            
            """Return JSON:
{{
    "annotations": [
        {{"question_label": "10(a)", "paper_pages": [5]}},
        {{"question_label": "10(b)", "paper_pages": [6, 7]}},
        ...
    ]
}}""",
            
            """CRITICAL:
- paper_pages is ALWAYS an array, even for single-page questions
- Page indices are 0-based
- Include ALL pages if question spans multiple pages
- Return annotations for ALL {total_questions} questions"""
        )
        
        # Solution pages annotator
        self.register(
            "annotate_solution",
            """You are analyzing a solution PDF with page markers.

The PDF has visible PAGE_INDEX_N markers (0-based) at the top-right of each page.""",
            
            """Here are the questions (you need to find their ANSWERS in this solution PDF):
{questions_list}""",
            
            """For EACH question, identify ALL pages where its ANSWER appears (may span multiple pages).""",
            
            """Return JSON:
{{
    "annotations": [
        {{"question_label": "10(a)", "solution_pages": [2]}},
        {{"question_label": "10(b)", "solution_pages": [3, 4]}},
        ...
    ]
}}""",
            
            """CRITICAL:
- solution_pages is ALWAYS an array, even for single-page answers
- Page indices are 0-based
- Include ALL pages if answer spans multiple pages
- Return annotations for ALL {total_questions} questions"""
        )
    
    # ============= Step 2: Question LaTeX Generator =============
    
    def _register_question_latex(self) -> None:
        """注册题目 LaTeX 生成 prompt"""
        self.register(
            "question_latex",
            """You are a professional LaTeX converter for exam questions.""",
            
            """=== Your Task ===
Convert question **{question_label}** from the PDF pages to clean, compilable LaTeX code.""",
            
            """=== Question Location ===
- Question Label: {question_label}
- Paper Pages: [{pages_str}] (0-based page indexing)
- **Note**: These page numbers are for reference. The actual question content may appear on nearby pages or span across adjacent pages.""",
            
            """=== Conversion Guidelines ===

1. **Read Question Content**:
   - The question is expected around page(s) {pages_str}
   - Check nearby pages if the question spans multiple pages or starts/ends on adjacent pages
   - Read ALL content across these pages
   - Include all sub-parts like (i), (ii), (iii) OR multiple choice options like A, B, C, D

2. **Convert to LaTeX**:
   - Use standard math environments: $...$ for inline, \\[...\\] or \\begin{{align*}}...\\end{{align*}} for display
   - Keep structure clear and organized
   - Use \\textbf{{}} for emphasis
   - Convert all mathematical symbols accurately
   - Preserve all formatting (fractions, powers, roots, etc.)
   - **For multiple choice questions (A, B, C, D options), use \\begin{{enumerate}}[label=\\Alph*.] format**

3. **Handle Images/Diagrams**:
   - If you see an image, graph, or diagram, note its position
   - Use placeholder: \\includegraphics[width=0.5\\textwidth]{{Figures/idPLACEHOLDER{question_index}_1.png}}
   - For multiple images, use: Figures/idPLACEHOLDER{question_index}_1.png, Figures/idPLACEHOLDER{question_index}_2.png, etc.
   - For each image, record:
     * page_number: which page the image appears on (0-based)
     * bbox: bounding box [x1, y1, x2, y2] (origin at top-left corner)
     * description: brief description of the image

4. **Formatting Rules**:
   - **MUST start with: \\item** (do NOT include the question label)
   - **For sub-parts (i), (ii), (iii), MUST use \\begin{{enumerate}}[label=(\\roman*)] and \\item**
   - **For multiple choice questions (A, B, C, D), MUST use \\begin{{enumerate}}[label=\\Alph*.] and \\item**
   - Keep consistent spacing
   - Don't add extra section titles

5. **Quality Check**:
   - Verify all math brackets match: (), [], \\{{\\}}
   - Check all LaTeX commands are spelled correctly
   - Ensure completeness - don't miss any part""",
            
            """=== Output Format ===
Return ONLY valid JSON (no markdown, no code blocks):
{{
    "question_label": "{question_label}",
    "question_latex": "...complete LaTeX code...",
    "question_images": [
        {{
            "page_number": 5,
            "bbox": [100.5, 200.3, 400.7, 500.2],
            "description": "Graph showing quadratic function"
        }}
    ],
    "compilation_success": true,
    "error_message": null
}}""",
            
            """=== Examples ===

Example 1 (simple):
{{
    "question_label": "10(a)",
    "question_latex": "\\\\item Solve the equation $x^2 + 3x - 4 = 0$.",
    "question_images": [],
    "compilation_success": true,
    "error_message": null
}}

Example 2 (with sub-parts):
{{
    "question_label": "Question 11",
    "question_latex": "\\\\item Consider the function $f(x) = x^2 - 4x + 3$.\\n\\\\begin{{enumerate}}[label=(\\\\roman*)]\\n\\\\item Find the vertex.\\n\\\\item Sketch the graph.\\n\\\\end{{enumerate}}",
    "question_images": [],
    "compilation_success": true,
    "error_message": null
}}

Example 3 (with images):
{{
    "question_label": "Question 8",
    "question_latex": "\\\\item The diagram shows a triangle ABC.\\n\\\\includegraphics[width=0.5\\\\textwidth]{{Figures/idPLACEHOLDER8_1.png}}\\n\\\\begin{{enumerate}}[label=(\\\\roman*)]\\n\\\\item Calculate the area.\\n\\\\item Find the perimeter.\\n\\\\end{{enumerate}}",
    "question_images": [
        {{
            "page_number": 3,
            "bbox": [150.0, 250.0, 450.0, 500.0],
            "description": "Triangle ABC with sides labeled: AB = 5cm, BC = 4cm, AC = 3cm"
        }}
    ],
    "compilation_success": true,
    "error_message": null
}}

Example 4 (multiple choice - MUST use this format):
{{
    "question_label": "Question 3",
    "question_latex": "\\\\item What is the derivative of $\\\\dfrac{{\\\\sin x}}{{e^x}}$?\\n\\\\begin{{enumerate}}[label=\\\\Alph*.]\\n\\\\item $\\\\dfrac{{\\\\sin x + \\\\cos x}}{{e^x}}$\\n\\\\item $\\\\dfrac{{\\\\sin x - \\\\cos x}}{{e^x}}$\\n\\\\item $-\\\\dfrac{{\\\\sin x + \\\\cos x}}{{e^x}}$\\n\\\\item $\\\\dfrac{{\\\\cos x - \\\\sin x}}{{e^x}}$\\n\\\\end{{enumerate}}",
    "question_images": [],
    "compilation_success": true,
    "error_message": null
}}

Now convert the question."""
        )
    
    # ============= Step 3: Answer LaTeX Generator =============
    
    def _register_answer_latex(self) -> None:
        """注册答案 LaTeX 生成 prompt"""
        self.register(
            "answer_latex",
            """You are a professional LaTeX converter for exam answers/solutions.""",
            
            """=== Your Task ===
Convert the answer for question **{question_label}** from the solution PDF to clean, compilable LaTeX code.""",
            
            """=== Answer Location ===
- Question Label: {question_label}
- Solution Pages: [{pages_str}] (0-based page indexing)
- **Note**: These page numbers are for reference. The actual answer content may appear on nearby pages or span across adjacent pages.""",
            
            """=== Conversion Guidelines ===

1. **Read Solution Content**:
   - The answer is expected around page(s) {pages_str}
   - Check nearby pages if the solution spans multiple pages or starts/ends on adjacent pages
   - Read ALL working and steps
   - Include complete solution process

2. **Convert to LaTeX**:
   - Show all working steps clearly
   - Use \\begin{{align*}}...\\end{{align*}} for multi-step calculations
   - Use \\therefore, \\implies for logical connections
   - Highlight final answer with \\boxed{{}} or \\textbf{{Answer:}}
   - Include text explanations between steps

3. **Extract Marks**:
   - Look for marks indicators: "[3 marks]", "(2m)", "3m", etc.
   - Extract the numerical value

4. **Handle Images**:
   - Note any solution diagrams or graphs
   - Use placeholder: \\includegraphics[width=0.5\\textwidth]{{Figures/idPLACEHOLDER{question_index}_sol_1.png}}
   - For multiple images, use: Figures/idPLACEHOLDER{question_index}_sol_1.png, Figures/idPLACEHOLDER{question_index}_sol_2.png, etc.
   - For each image, record:
     * page_number: which page the image appears on (0-based)
     * bbox: bounding box [x1, y1, x2, y2] (origin at top-left corner)
     * description: brief description of the image

5. **Formatting Rules**:
   - **MUST start with: \\item** (do NOT include the question label)
   - **For answers with sub-parts (i), (ii), (iii), MUST use \\begin{{enumerate}}[label=(\\roman*)] and \\item for each sub-part**
   - Clear step-by-step presentation
   - Use \\text{{}} for English within math mode
   - Show intermediate steps
   - Emphasize final answer""",
            
            """=== Output Format ===
Return ONLY valid JSON:
{{
    "question_label": "{question_label}",
    "answer_latex": "...complete LaTeX solution...",
    "answer_images": [
        {{
            "page_number": 0,
            "bbox": [100.5, 200.3, 400.7, 500.2],
            "description": "Solution diagram showing triangles"
        }}
    ],
    "marks": 3,
    "compilation_success": true,
    "error_message": null
}}""",
            
            """=== Examples ===

Example 1 (without images):
{{
    "question_label": "10(a)",
    "answer_latex": "\\\\item \\\\begin{{align*}}\\nx^2 + 3x - 4 &= 0 \\\\\\\\\\n(x + 4)(x - 1) &= 0 \\\\\\\\\\nx &= -4 \\\\text{{ or }} x = 1\\n\\\\end{{align*}}\\n\\\\textbf{{Answer:}} $x = -4$ or $x = 1$",
    "answer_images": [],
    "marks": 2,
    "compilation_success": true,
    "error_message": null
}}

Example 2 (with images):
{{
    "question_label": "Question 5",
    "answer_latex": "\\\\item \\\\includegraphics[width=0.5\\\\textwidth]{{Figures/idPLACEHOLDER5_sol_1.png}}\\n\\\\begin{{align*}}\\nArea &= \\\\frac{{1}}{{2}} \\\\times base \\\\times height \\\\\\\\\\n&= \\\\frac{{1}}{{2}} \\\\times 4 \\\\times 3 \\\\\\\\\\n&= 6 \\\\text{{ cm}}^2\\n\\\\end{{align*}}",
    "answer_images": [
        {{
            "page_number": 35,
            "bbox": [50.0, 100.0, 300.0, 350.0],
            "description": "Diagram showing triangle with labeled sides"
        }}
    ],
    "marks": 3,
    "compilation_success": true,
    "error_message": null
}}

Now convert the answer."""
        )
    
    # ============= Step 4: Bbox Corrector =============
    
    def _register_bbox_corrector(self) -> None:
        """注册 bbox 修正 prompt"""
        self.register(
            "bbox_correction",
            """You are an expert at validating image crops from PDF documents.""",
            
            """=== Your Task ===
Verify if the cropped image correctly captures the {image_type} content for {question_label}.""",
            
            """=== Current Crop Info ===
- Question Label: {question_label}
- Current BBox (PDF points): {current_bbox} (format: [x1, y1, x2, y2])
- Current BBox (rendered pixels): [{bbox_rendered_str}]
- Expected Content: {expected_description}
- Image Type: {image_type}""",
            
            """=== Dimension Information ===
- PDF Page Size: {pdf_width:.1f} x {pdf_height:.1f} points (PDF coordinate system)
- Rendered Page Size: {rendered_width} x {rendered_height} pixels
- Cropped Image Size: {cropped_width} x {cropped_height} pixels
- Scale Factor: {scale_x:.3f} (X), {scale_y:.3f} (Y) pixels per PDF point""",
            
            """=== Important Context ===
The expected description may include:
- **For multiple choice questions**: The description may indicate which option the image belongs to (e.g., "Image for option A", "Diagram for choice B")
- **For sub-parts**: The description may indicate which sub-part the image belongs to (e.g., "Image for part (i)", "Diagram for part (ii)")
- **General descriptions**: The description may describe the image content (e.g., "Graph showing quadratic function", "Triangle diagram")

When validating, pay special attention to:
- If the description mentions an option (A, B, C, D), verify the crop includes ONLY that option's image
- If the description mentions a sub-part (i, ii, iii), verify the crop includes ONLY that sub-part's image
- Ensure the crop matches the specific context mentioned in the description""",
            
            """=== Evaluation Criteria ===
1. **Completeness**: Is all relevant content included?
2. **Accuracy**: Is the content correctly identified and matches the expected description?
3. **Context Match**: Does the crop match the specific context (option, sub-part, etc.) mentioned in the description?
4. **Boundaries**: Are the boundaries appropriate (not too tight/loose)?
5. **No Extra Content**: Is there minimal irrelevant content? For option-specific images, ensure no other options are included.""",
            
            """=== Instructions ===
1. Compare the cropped image with the rendered PDF page image
2. Check if the crop matches the expected description, paying special attention to context (option, sub-part, etc.)
3. Verify the crop includes only the relevant content mentioned in the description
4. If incorrect, provide corrected coordinates in PDF points (not pixels)
5. PDF coordinate system: origin at top-left, x increases right, y increases down
6. Coordinate conversion: PDF points × scale_factor = rendered pixels
7. The rendered page image shows the full PDF page, use it as reference to identify the correct crop area""",
            
            """=== Common Issues to Check ===
- Coordinates might be scaled incorrectly (e.g., pixel coordinates vs PDF points)
- Crop might be too small or too large
- Crop might be offset from the actual content
- Content might span a larger area than the current bbox""",
            
            """=== Output Format ===
Return ONLY valid JSON:
{{
    "is_correct": true/false,
    "confidence": 0.95,
    "issue_description": "Crop is too narrow, missing right side of diagram" or null,
    "corrected_bbox": [x1, y1, x2, y2] or null,
    "reasoning": "Detailed explanation of your decision..."
}}

If crop is correct, set corrected_bbox to null.
If crop needs correction, provide the corrected coordinates in PDF points.

Now analyze the cropped image and the PDF page."""
        )
    
    # ============= Step 5: Labelling =============
    
    def _register_labelling(self) -> None:
        """注册题目标注 prompt"""
        self.register(
            "labelling",
            """You are a Question Labelling Agent. Your task is to analyze a question and label it with accurate metadata.""",
            
            """=== Question Information ===
Question Index: {question_index}
Question Label: {question_label}""",
            
            """=== Question Content ===
{question_latex}
{answer_section}""",
            
            """=== Your Task ===

You need to label this question with the following metadata:

1. **Topic and Subtopic** (MOST IMPORTANT):
   - You MUST select the MOST ACCURATE subtopic from the provided list below
   - You CANNOT create new topics or subtopics - you MUST choose from the list
   - The subtopic_id is the MOST CRITICAL field - it must be accurate
   - Provide a confidence score (0.0-1.0) for your subtopic selection
   - If you are uncertain, explain why in the reasoning field

2. **Question Type** (REQUIRED):
   - You MUST choose EXACTLY ONE from: "short answer" OR "multiple choice"
   - **Multiple Choice**: Has explicit options (A, B, C, D, etc.), usually with instructions like "circle", "select", "choose"
   - **Short Answer**: Requires students to write their answer, may have blank lines, underscores, or answer spaces
   - Look at the question structure and answer format to determine the type

3. **Difficulty** (OPTIONAL):
   - Assess the difficulty based on:
     * Complexity of concepts involved
     * Number of steps required to solve
     * Level of mathematical reasoning needed
   - Common values: "Easy", "Medium", "Hard", or specific difficulty levels
   - If uncertain, you can leave it as null

4. **Mark** (OPTIONAL):{mark_instruction}""",
            
            """=== Available Topics and Subtopics ===

You MUST select from this list (DO NOT create new ones):

{subtopics_text}

**CRITICAL RULES**:
- You MUST select a subtopic_id from the list above
- The subtopic_id is the MOST IMPORTANT field - accuracy is critical
- If no subtopic matches perfectly, choose the CLOSEST match and explain in reasoning
- Provide confidence score for your subtopic selection""",
            
            """=== Output Format ===

Return ONLY valid JSON (no markdown, no code blocks):
{{
    "question_index": {question_index},
    "question_label": "{question_label}",
    "topic_id": <integer>,
    "subtopic_id": <integer>,
    "question_type": "short answer" or "multiple choice",
    "difficulty": "<string>" or null,
    "mark": <integer> or null,
    "confidence": <float between 0.0 and 1.0>,
    "reasoning": "<detailed explanation of your decisions, especially for subtopic selection>"
}}""",
            
            """=== Examples ===

Example 1 (Multiple Choice):
{{
    "question_index": 3,
    "question_label": "Question 3",
    "topic_id": 15,
    "subtopic_id": 42,
    "question_type": "multiple choice",
    "difficulty": "Medium",
    "mark": 2,
    "confidence": 0.95,
    "reasoning": "This is a multiple choice question about derivatives. The subtopic 'Derivatives of Trigonometric Functions' (subtopic_id: 42) is the most accurate match. The question has 4 options (A, B, C, D) and asks to select the correct answer."
}}

Example 2 (Short Answer):
{{
    "question_index": 10,
    "question_label": "10(a)",
    "topic_id": 12,
    "subtopic_id": 28,
    "question_type": "short answer",
    "difficulty": "Hard",
    "mark": 5,
    "confidence": 0.88,
    "reasoning": "This is a short answer question about quadratic equations. The subtopic 'Solving Quadratic Equations' (subtopic_id: 28) matches well. The question requires students to show their working and write the answer. The difficulty is high because it involves completing the square method."
}}

Now analyze the question and provide the labels."""
        )
    
    # ============= 辅助方法 =============
    
    def build_labelling_prompt(
        self,
        question_index: int,
        question_label: str,
        question_latex: str,
        answer_latex: Optional[str],
        subtopics_list: List[dict],
        existing_mark: Optional[int] = None
    ) -> str:
        """构建 labelling prompt（特殊处理子主题列表和分数）"""
        # 构建 subtopic 选项列表
        subtopics_text = "\n".join([
            f"  {idx + 1}. Topic: {s.get('topic_name') or s.get('topicname', 'N/A')} "
            f"(topic_id: {s.get('topicid') or s.get('topic_id', 'N/A')}) | "
            f"Subtopic: {s.get('subtopic_name') or s.get('subtopicname', 'N/A')} "
            f"(subtopic_id: {s.get('subtopicid') or s.get('subtopic_id', 'N/A')})"
            for idx, s in enumerate(subtopics_list)
        ])
        
        # Mark 说明
        if existing_mark is not None:
            mark_instruction = (
                f"\n- **Mark**: The question already has a mark of {existing_mark}. "
                f"Verify if this is correct based on the question content. If incorrect, extract the correct mark."
            )
        else:
            mark_instruction = (
                "\n- **Mark**: Extract the mark from the question "
                "(look for notations like [5], [8 marks], etc.). If not found, leave as null."
            )
        
        # Answer section
        answer_section = ""
        if answer_latex:
            answer_section = f"""
=== Answer Content ===
{answer_latex}

**Note**: The answer content can help you understand the question better and determine its difficulty.
"""
        
        return self.build(
            "labelling",
            question_index=question_index,
            question_label=question_label,
            question_latex=question_latex,
            answer_section=answer_section,
            subtopics_text=subtopics_text,
            mark_instruction=mark_instruction
        )
    
    def get_all_steps(self) -> List[str]:
        """获取所有已注册的步骤名称"""
        return [key for key in self._sections.keys() if not key.startswith("_")]


# ============= 全局单例 =============

PROMPTS = PromptRegistry()
PROMPTS.register_all()


# ============= 使用示例 =============

if __name__ == "__main__":
    # 示例 1: Classifier
    classifier_prompt = PROMPTS.build("classify")
    print("=== CLASSIFIER PROMPT ===")
    print(classifier_prompt[:500])
    print("\n")
    
    # 示例 2: Lister (Type1)
    lister_prompt = PROMPTS.build_lister_prompt(exam_type="type1", emphasize=False)
    print("=== LISTER PROMPT (Type1) ===")
    print(lister_prompt[:500])
    print("\n")

=== CLASSIFIER PROMPT ===
Analyze the provided pages of this exam and determine its type.

**Type1** (Separate Answer Booklet):
- Explicitly states "Use a SEPARATE writing booklet" or similar
- No blank lines or answer spaces under questions
- Questions are densely packed
- Example: "10(a)", "10(b)", "10(c)" are independent questions

**Type2** (Answer on Paper):
- Has blank lines or answer spaces under questions, including:
  * Underscores (______)
  * Dotted lines (..................)
  * Multiple blank lines for writi


=== LISTER PROMPT (Type1) ===
You are a Question Lister Agent. Your task is to scan the entire paper PDF and create a **complete, accurate list** of all questions.

=== Exam Type ===
type1

=== Question Splitting Rules ===
【Type1 Rules】(Separate Answer Booklet):
- Question 10 is a **section title**, not an independent question
- 10(a), 10(b), 10(c) are **independent questions** (minimum splitting unit)
- 10(c)(i), 10(c)(ii) are **sub-parts** of 10(c), NOT separate qu

In [8]:
# llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)

# agent = create_agent(
#     model=llm,
#     tools=[],
#     system_prompt="You are an Australian high school teacher.",
# )

# # === 3. 构造消息并执行 ===
# question = HumanMessage(content=[
#     {"type": "text", "text":
#         "You are an Australian high school teacher.\n"
#         "Please ONLY summarize the content of the third page of the solution PDF. Do NOT scan the whole PDF. "
#     },
#     {"type": "media", "mime_type": "application/pdf", "file_uri": q_uri},
#     {"type": "media", "mime_type": "application/pdf", "file_uri": s_uri},
# ])


In [9]:
from langchain.agents.middleware import AgentMiddleware, ModelRequest, ModelResponse
from typing import Callable, Awaitable

class LongPromptMiddleware(AgentMiddleware):
    """
    核心中间件：根据 FlowContext 动态组装 prompt、裁剪 token、控制工具
    """
    
    def wrap_model_call(
        self,
        request: ModelRequest,
        handler: Callable[[ModelRequest], ModelResponse],
    ) -> ModelResponse:
        """同步版本"""
        ctx: FlowContext = request.runtime.context
        
        # 1) 根据步骤构建长 prompt
        if ctx.step == "classify":
            long_system = PROMPTS.build("classify")
        elif ctx.step == "lister":
            long_system = PROMPTS.build_lister_prompt(
                exam_type=ctx.exam_type,
                emphasize=(ctx.retry_count > 0)
            )
        elif ctx.step == "question_latex":
            long_system = PROMPTS.build(
                "question_latex",
                question_label=ctx.current_question_label,
                pages_str=", ".join(map(str, ctx.paper_pages or [])),
                question_index=ctx.current_question_index
            )
        elif ctx.step == "answer_latex":
            long_system = PROMPTS.build(
                "answer_latex",
                question_label=ctx.current_question_label,
                pages_str=", ".join(map(str, ctx.solution_pages or [])),
                question_index=ctx.current_question_index
            )
        else:
            long_system = PROMPTS.build(ctx.step)
        
        # 2) Token 裁剪
        def rough_token_count(txt: str) -> int:
            return max(1, len(txt) // 4)
        
        if rough_token_count(long_system) > ctx.token_budget:
            long_system = long_system[:ctx.token_budget * 4]
        
        # 3) 注入 system prompt
        if request.system_prompt:
            request.system_prompt = f"{long_system}\n\n{request.system_prompt}"
        else:
            request.system_prompt = long_system
        
        # 4) 工具白名单控制
        step_config = ctx.get_step_config()
        if not step_config.get("enable_tools", True):
            request.tools = []
        
        # 5) 调用模型
        response = handler(request)
        return response
    
    async def awrap_model_call(
        self,
        request: ModelRequest,
        handler: Callable[[ModelRequest], Awaitable[ModelResponse]],
    ) -> ModelResponse:
        """异步版本 ⭐ 这是修复 NotImplementedError 的关键"""
        ctx: FlowContext = request.runtime.context
        
        # 1) 根据步骤构建长 prompt
        if ctx.step == "classify":
            long_system = PROMPTS.build("classify")
        elif ctx.step == "lister":
            long_system = PROMPTS.build_lister_prompt(
                exam_type=ctx.exam_type,
                emphasize=(ctx.retry_count > 0)
            )
        elif ctx.step == "question_latex":
            long_system = PROMPTS.build(
                "question_latex",
                question_label=ctx.current_question_label,
                pages_str=", ".join(map(str, ctx.paper_pages or [])),
                question_index=ctx.current_question_index
            )
        elif ctx.step == "answer_latex":
            long_system = PROMPTS.build(
                "answer_latex",
                question_label=ctx.current_question_label,
                pages_str=", ".join(map(str, ctx.solution_pages or [])),
                question_index=ctx.current_question_index
            )
        else:
            long_system = PROMPTS.build(ctx.step)
        
        # 2) Token 裁剪
        def rough_token_count(txt: str) -> int:
            return max(1, len(txt) // 4)
        
        if rough_token_count(long_system) > ctx.token_budget:
            long_system = long_system[:ctx.token_budget * 4]
        
        # 3) 注入 system prompt
        if request.system_prompt:
            request.system_prompt = f"{long_system}\n\n{request.system_prompt}"
        else:
            request.system_prompt = long_system
        
        # 4) 工具白名单控制
        step_config = ctx.get_step_config()
        if not step_config.get("enable_tools", True):
            request.tools = []
        
        # 5) 调用模型（异步 await）
        response = await handler(request)
        return response


class OutputValidationMiddleware(AgentMiddleware):
    """输出校验中间件：验证 JSON 格式、必填字段等"""
    
    def wrap_model_call(
        self,
        request: ModelRequest,
        handler: Callable[[ModelRequest], ModelResponse],
    ) -> ModelResponse:
        """同步版本"""
        ctx: FlowContext = request.runtime.context
        response = handler(request)
        
        # JSON 校验 - 支持 markdown 格式
        if ctx.response_format == "json_object":
            if response.result and len(response.result) > 0:
                last_message = response.result[-1]
                
                if hasattr(last_message, 'content') and isinstance(last_message.content, str):
                    try:
                        import json
                        import re
                        
                        content = last_message.content.strip()
                        # 提取 markdown 代码块中的 JSON
                        code_block_pattern = r'```(?:json)?\s*\n?(.*?)\n?```'
                        match = re.search(code_block_pattern, content, re.DOTALL)
                        json_str = match.group(1).strip() if match else content
                        
                        json.loads(json_str)
                        print(f"✅ JSON 格式验证通过 (步骤: {ctx.step})")
                    except json.JSONDecodeError as e:
                        print(f"⚠️ JSON 解析失败 (步骤: {ctx.step}): {e}")
                        print(f"   内容: {last_message.content[:200]}...")
                else:
                    print(f"ℹ️ 最后一条消息不是纯文本 (步骤: {ctx.step})，跳过 JSON 校验")
        
        return response
    
    async def awrap_model_call(
        self,
        request: ModelRequest,
        handler: Callable[[ModelRequest], Awaitable[ModelResponse]],
    ) -> ModelResponse:
        """异步版本 ⭐"""
        ctx: FlowContext = request.runtime.context
        response = await handler(request)
        
        # JSON 校验 - 支持 markdown 格式
        if ctx.response_format == "json_object":
            if response.result and len(response.result) > 0:
                last_message = response.result[-1]
                
                if hasattr(last_message, 'content') and isinstance(last_message.content, str):
                    try:
                        import json
                        import re
                        
                        content = last_message.content.strip()
                        # 提取 markdown 代码块中的 JSON
                        code_block_pattern = r'```(?:json)?\s*\n?(.*?)\n?```'
                        match = re.search(code_block_pattern, content, re.DOTALL)
                        json_str = match.group(1).strip() if match else content
                        
                        json.loads(json_str)
                        print(f"✅ JSON 格式验证通过 (步骤: {ctx.step})")
                    except json.JSONDecodeError as e:
                        print(f"⚠️ JSON 解析失败 (步骤: {ctx.step}): {e}")
                        print(f"   内容: {last_message.content[:200]}...")
                else:
                    print(f"ℹ️ 最后一条消息不是纯文本 (步骤: {ctx.step})，跳过 JSON 校验")
        
        return response

In [10]:
import json
import re

def extract_json_from_markdown(content: str) -> dict:
    """
    从模型返回的内容中提取 JSON（处理 markdown 代码块）
    
    支持格式：
    1. ```json {...} ```
    2. ``` {...} ```
    3. 纯 JSON {...}
    """
    # 去除首尾空白
    content = content.strip()
    
    # 尝试提取 markdown 代码块中的 JSON
    # 匹配 ```json ... ``` 或 ``` ... ```
    code_block_pattern = r'```(?:json)?\s*\n?(.*?)\n?```'
    match = re.search(code_block_pattern, content, re.DOTALL)
    
    if match:
        # 提取代码块内的内容
        json_str = match.group(1).strip()
    else:
        # 没有代码块，直接使用原内容
        json_str = content
    
    # 尝试解析 JSON
    try:
        return json.loads(json_str)
    except json.JSONDecodeError as e:
        # 如果失败，尝试找到第一个 { 和最后一个 }
        start = json_str.find('{')
        end = json_str.rfind('}')
        
        if start != -1 and end != -1 and end > start:
            json_str = json_str[start:end+1]
            return json.loads(json_str)
        else:
            raise ValueError(f"无法从内容中提取 JSON: {content[:200]}...") from e

# 测试
test_cases = [
    '```json\n{"exam_type": "type1"}\n```',
    '```\n{"exam_type": "type2"}\n```',
    '{"exam_type": "type1"}',
    'Here is the result:\n```json\n{"exam_type": "type1"}\n```\nDone.'
]

for i, test in enumerate(test_cases):
    try:
        result = extract_json_from_markdown(test)
        print(f"✅ Test {i+1}: {result}")
    except Exception as e:
        print(f"❌ Test {i+1}: {e}")


✅ Test 1: {'exam_type': 'type1'}
✅ Test 2: {'exam_type': 'type2'}
✅ Test 3: {'exam_type': 'type1'}
✅ Test 4: {'exam_type': 'type1'}


In [11]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent

# 1. 创建 LLM
llm = ChatOpenAI(
    model="gpt-5",  # ✅ 修复：使用正确的模型名
    temperature=0,
)

# 2. 定义工具（如果需要 file_search）
from langchain_core.tools import tool

@tool
def file_search(query: str, file_id: str) -> str:
    """Search the PDF file by keyword/pattern"""
    # ⭐ OpenAI 的 file search 需要使用 Assistants API
    # 这里简化为示例
    return f"Searched file {file_id} for: {query}"

TOOLS = [file_search]

# 3. 创建 Agent（带中间件）
agent = create_agent(
    model=llm,
    tools=TOOLS,
    middleware=[
        LongPromptMiddleware(),       # 动态 prompt 注入
        OutputValidationMiddleware()  # JSON 校验
    ],
    context_schema=FlowContext  # 允许传入 FlowContext
)

In [12]:
# 在 notebook 新增 cell
from langchain.messages import HumanMessage

async def run_classify_step(
    paper_file_id: str,  
    solution_file_id: str,  
    exam_id: str
) -> dict:
    """Step 0: 分类试卷类型"""
    
    # 1. 创建 context
    context = FlowContext(
        step="classify",
        exam_id=exam_id,
        paper_file_id=paper_file_id, 
        solution_file_id=solution_file_id, 
        token_budget=8000,
        response_format="json_object"
    )
    
    # 2. 构建消息（⭐ OpenAI 格式）
    message = HumanMessage(content=[
        {"type": "text", "text": "Analyze the exam type from these pages."},
        {"type": "file", "file_id": paper_file_id} 
    ])
    
    # 3. 调用 agent（传入 context）
    result = await agent.ainvoke(
        {"messages": [message]},
        context=context
    )
    
    # 4. 解析结果（使用 extract_json_from_markdown 处理 markdown 代码块）
    content = result["messages"][-1].content
    output = extract_json_from_markdown(content)
    
    print(f"✅ 分类完成: {output['exam_type']}")
    print(f"   理由: {output['reasoning']}")
    
    return {
        "exam_type": output["exam_type"],
        "reasoning": output["reasoning"],
        "confidence": output.get("confidence")
    }

async def run_lister_step(
    paper_file_id: str,
    exam_type: str,
    exam_id: str
) -> dict:
    """Step 1: 列出所有题目"""
    
    context = FlowContext(
        step="lister",
        exam_id=exam_id,
        exam_type=exam_type,
        paper_file_id=paper_file_id,
        token_budget=16000,
        max_tokens=16000,
        response_format="json_object"
    )
    
    message = HumanMessage(content=[
        {"type": "text", "text": "List all questions from the paper PDF."},
        {"type": "file", "file_id": paper_file_id} 
    ])
    
    result = await agent.ainvoke(
        {"messages": [message]},
        context=context
    )
    
    # 解析结果
    content = result["messages"][-1].content
    output = extract_json_from_markdown(content)
    
    print(f"✅ 列题完成: {output['total_questions']} 道题")
    print(f"   前 3 题: {[q['question_label'] for q in output['questions'][:3]]}")
    
    return output


async def run_question_latex_step(
    paper_file_id: str,
    question_index: int,
    question_label: str,
    paper_pages: list,
    exam_id: str
) -> dict:
    """Step 2: 生成题目 LaTeX"""
    
    context = FlowContext(
        step="question_latex",
        exam_id=exam_id,
        current_question_index=question_index,
        current_question_label=question_label,
        paper_pages=paper_pages,
        paper_file_id=paper_file_id,
        token_budget=12000,
        max_tokens=8000,
        response_format="json_object"
    )
    
    message = HumanMessage(content=[
        {"type": "text", "text": f"Convert question {question_label} to LaTeX."},
        {"type": "file", "file_id": paper_file_id} 
    ])
    
    result = await agent.ainvoke(
        {"messages": [message]},
        context=context
    )
    
    # 解析结果
    content = result["messages"][-1].content
    output = extract_json_from_markdown(content)
    
    print(f"✅ 题目 LaTeX 完成: {question_label}")
    print(f"   LaTeX 长度: {len(output['question_latex'])} 字符")
    print(f"   图片数: {len(output.get('question_images', []))}")
    
    return output

In [ ]:
# 在 notebook 新增 cell
async def run_complete_workflow(
    paper_file_id: str,
    solution_file_id: str,
    exam_id: str = "exam_001"
):
    """完整流程：分类 → 列题 → 生成 LaTeX"""
    
    print("="*60)
    print(f"🚀 开始处理试卷: {exam_id}")
    print("="*60)
    
    # Step 0: 分类
    print("\n📊 Step 0: 分类试卷类型...")
    classify_result = await run_classify_step(paper_file_id, solution_file_id, exam_id)
    exam_type = classify_result["exam_type"]
    
    # Step 1: 列题
    print(f"\n📋 Step 1: 列出所有题目 (类型: {exam_type})...")
    lister_result = await run_lister_step(paper_file_id, exam_type, exam_id)
    questions = lister_result["questions"]
    
    # Step 2: 对每道题生成 LaTeX（示例：只处理前 3 题）
    print(f"\n📝 Step 2: 生成题目 LaTeX (处理前 3 题)...")
    latex_results = []
    for q in questions[:14]:
        latex_result = await run_question_latex_step(
            paper_file_id=paper_file_id,
            question_index=q["question_index"],
            question_label=q["question_label"],
            paper_pages=[0],  # 实际应从 annotate 步骤获取
            exam_id=exam_id
        )
        latex_results.append(latex_result)
    
    print("\n" + "="*60)
    print("✅ 处理完成！")
    print("="*60)
    
    return {
        "exam_id": exam_id,
        "exam_type": exam_type,
        "total_questions": lister_result["total_questions"],
        "latex_results": latex_results
    }


# 运行完整流程
result = await run_complete_workflow(paper_file_id, solution_file_id, "exam_test_001")

🚀 开始处理试卷: exam_test_001

📊 Step 0: 分类试卷类型...
✅ JSON 格式验证通过 (步骤: classify)
✅ 分类完成: type1
   理由: Multiple indicators point to a separate answer booklet format. (1) Explicit instructions: Section I (page 2/15) says 'Use the multiple-choice answer sheet for Questions 1–10.' Section II introduction (page 7/15) states 'Answer each question in the appropriate writing booklet. Extra writing booklets are available.' Each long-response question explicitly says 'Use a SEPARATE writing booklet.' (e.g., Question 11 on page 7/15, Question 12 on page 8/15, Question 13 on page 11/15, Question 14 on page 13/15). (2) No blank lines or answer spaces appear under the questions; content is densely packed across pages. (3) The provided sketch sheet for Q12(c) (page 15/15) is an insert to 'Place inside booklet for Q12,' reinforcing the separate booklet usage rather than answering on the paper itself. (4) Numbering and layout follow typical exam format with multi-part questions but without embedded answer are

In [ ]:
# ============= 查看生成的内容 =============

# 1. 查看分类结果
print("\n" + "="*60)
print("📊 分类结果")
print("="*60)
print(f"试卷类型: {result['exam_type']}")
print(f"题目总数: {result['total_questions']}")

# 2. 查看生成的 LaTeX（前 3 题）
print("\n" + "="*60)
print("📝 生成的 LaTeX 内容")
print("="*60)

for i, latex_result in enumerate(result['latex_results'], 1):
    print(f"\n{'='*60}")
    print(f"Question {i}")
    print(f"{'='*60}")
    print(f"\n{latex_result['question_latex']}\n")
    
    if latex_result.get('question_images'):
        print(f"📷 图片数: {len(latex_result['question_images'])}")
        for img in latex_result['question_images']:
            print(f"  - {img}")

# 3. 保存结果到文件
import json
from datetime import datetime

output_dir = "./output"
os.makedirs(output_dir, exist_ok=True)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_file = f"{output_dir}/exam_{result['exam_id']}_{timestamp}.json"

with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(result, f, indent=2, ensure_ascii=False)

print(f"\n✅ 结果已保存到: {output_file}")

# 4. 生成 LaTeX 预览文件
latex_preview_file = f"{output_dir}/exam_{result['exam_id']}_{timestamp}.tex"

with open(latex_preview_file, 'w', encoding='utf-8') as f:
    f.write("\\documentclass{article}\n")
    f.write("\\usepackage{amsmath}\n")
    f.write("\\usepackage{graphicx}\n")
    f.write("\\begin{document}\n\n")
    f.write(f"\\section*{{Exam: {result['exam_id']}}}\n")
    f.write(f"Type: {result['exam_type']}, Total Questions: {result['total_questions']}\n\n")
    
    for i, latex_result in enumerate(result['latex_results'], 1):
        f.write(f"\\subsection*{{Question {i}}}\n")
        f.write(latex_result['question_latex'])
        f.write("\n\n")
    
    f.write("\\end{document}\n")

print(f"✅ LaTeX 预览已保存到: {latex_preview_file}")
print(f"\n💡 提示: 可以用以下命令编译 LaTeX:")
print(f"   cd {output_dir} && pdflatex {os.path.basename(latex_preview_file)}")

In [ ]:
# ============= 快速查看工具 =============

def show_result_summary(result: dict):
    """显示结果摘要"""
    print("="*70)
    print(f"📋 试卷摘要: {result['exam_id']}")
    print("="*70)
    print(f"类型: {result['exam_type']}")
    print(f"总题数: {result['total_questions']}")
    print(f"已生成 LaTeX: {len(result['latex_results'])} 题")
    print()

def show_question_latex(result: dict, question_num: int):
    """显示指定题目的 LaTeX"""
    if question_num < 1 or question_num > len(result['latex_results']):
        print(f"❌ 题号超出范围 (1-{len(result['latex_results'])})")
        return
    
    latex_result = result['latex_results'][question_num - 1]
    
    print("="*70)
    print(f"📝 Question {question_num} LaTeX")
    print("="*70)
    print(latex_result['question_latex'])
    print()
    
    if latex_result.get('question_images'):
        print(f"📷 图片: {len(latex_result['question_images'])} 个")
        for img in latex_result['question_images']:
            print(f"  - {img}")
    print()

def compare_questions(result: dict):
    """对比所有题目的 LaTeX 长度"""
    print("="*70)
    print("📊 题目长度统计")
    print("="*70)
    
    for i, latex_result in enumerate(result['latex_results'], 1):
        latex_len = len(latex_result['question_latex'])
        img_count = len(latex_result.get('question_images', []))
        
        print(f"Q{i}: {latex_len:4d} 字符, {img_count} 图片")
    print()

# 使用示例
show_result_summary(result)
compare_questions(result)

# 查看第 1 题的详细内容
print("\n")
show_question_latex(result, 1)


In [ ]:
# ============= 提取和保存图片 =============

import fitz  # PyMuPDF
from PIL import Image
import io

def extract_images_from_pdf(pdf_path: str, result: dict, output_dir: str = "./output/images"):
    """
    根据结果中的图片信息，从 PDF 中提取图片
    
    Args:
        pdf_path: PDF 文件路径
        result: workflow 生成的结果
        output_dir: 图片保存目录
    """
    os.makedirs(output_dir, exist_ok=True)
    
    # 打开 PDF
    doc = fitz.open(pdf_path)
    
    extracted_images = []
    total_images = 0
    
    print("="*70)
    print("📷 提取图片中...")
    print("="*70)
    
    for q_idx, latex_result in enumerate(result['latex_results'], 1):
        if not latex_result.get('question_images'):
            continue
        
        for img_idx, img_info in enumerate(latex_result['question_images']):
            total_images += 1
            
            page_num = img_info['page_number']
            bbox = img_info['bbox']  # [x0, y0, x1, y1]
            
            # 读取页面
            page = doc[page_num]
            
            # 提取区域的像素图（bbox 格式: x0, y0, x1, y1）
            rect = fitz.Rect(bbox[0], bbox[1], bbox[2], bbox[3])
            
            # 设置缩放比例（提高分辨率）
            mat = fitz.Matrix(2, 2)  # 2倍放大
            pix = page.get_pixmap(matrix=mat, clip=rect)
            
            # 保存图片
            img_filename = f"q{q_idx}_img{img_idx+1}_p{page_num}.png"
            img_path = os.path.join(output_dir, img_filename)
            pix.save(img_path)
            
            extracted_images.append({
                'question': q_idx,
                'image_index': img_idx + 1,
                'page': page_num,
                'filename': img_filename,
                'path': img_path,
                'bbox': bbox
            })
            
            print(f"✅ Q{q_idx} 图片 {img_idx+1}: {img_filename} (Page {page_num})")
    
    doc.close()
    
    print(f"\n{'='*70}")
    print(f"✅ 提取完成！共 {total_images} 张图片")
    print(f"📁 保存位置: {output_dir}")
    print("="*70)
    
    return extracted_images


def update_latex_with_images(result: dict, images_info: list, output_dir: str = "./output"):
    """
    更新 LaTeX 文件，将 PLACEHOLDER 替换为实际的图片路径
    """
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    latex_file = f"{output_dir}/exam_{result['exam_id']}_with_images_{timestamp}.tex"
    
    with open(latex_file, 'w', encoding='utf-8') as f:
        f.write("\\documentclass{article}\n")
        f.write("\\usepackage{amsmath}\n")
        f.write("\\usepackage{graphicx}\n")
        f.write("\\usepackage{enumitem}\n")
        f.write("\\begin{document}\n\n")
        f.write(f"\\section*{{Exam: {result['exam_id']}}}\n")
        f.write(f"Type: {result['exam_type']}, Total Questions: {result['total_questions']}\n\n")
        
        f.write("\\begin{enumerate}\n")
        
        for q_idx, latex_result in enumerate(result['latex_results'], 1):
            latex_content = latex_result['question_latex']
            
            # 替换 PLACEHOLDER 为实际图片路径
            if latex_result.get('question_images'):
                for img_idx, img_info in enumerate(latex_result['question_images']):
                    placeholder = f"idPLACEHOLDER{q_idx}_{img_idx+1}"
                    actual_filename = f"images/q{q_idx}_img{img_idx+1}_p{img_info['page_number']}.png"
                    latex_content = latex_content.replace(
                        f"Figures/{placeholder}.png",
                        actual_filename
                    )
            
            f.write(latex_content)
            f.write("\n\n")
        
        f.write("\\end{enumerate}\n")
        f.write("\\end{document}\n")
    
    print(f"\n✅ 更新的 LaTeX 已保存到: {latex_file}")
    print(f"\n💡 编译命令:")
    print(f"   cd {output_dir} && pdflatex {os.path.basename(latex_file)}")
    
    return latex_file


# 使用示例
print("\n" + "="*70)
print("开始提取图片...")
print("="*70)

# 提取图片
images_info = extract_images_from_pdf(paper_pdf_path, result)

# 更新 LaTeX 文件
if images_info:
    latex_file = update_latex_with_images(result, images_info)
else:
    print("\nℹ️ 没有需要提取的图片")

# 显示图片信息摘要
if images_info:
    print(f"\n📊 图片摘要:")
    for img in images_info:
        print(f"  Q{img['question']} - {img['filename']}")


In [ ]:
# ============= 在 Notebook 中显示图片 =============

from IPython.display import display, Image as IPImage, HTML

def show_extracted_images(images_info: list, max_width: int = 600):
    """在 notebook 中显示提取的图片"""
    if not images_info:
        print("ℹ️ 没有图片可显示")
        return
    
    print("="*70)
    print(f"📷 显示 {len(images_info)} 张提取的图片")
    print("="*70)
    
    for img in images_info:
        print(f"\n{'─'*70}")
        print(f"Question {img['question']} - 图片 {img['image_index']} (Page {img['page']})")
        print(f"文件: {img['filename']}")
        print(f"{'─'*70}")
        
        # 显示图片
        display(IPImage(filename=img['path'], width=max_width))


def show_question_with_images(result: dict, question_num: int, images_info: list):
    """显示题目的 LaTeX 和对应的图片"""
    if question_num < 1 or question_num > len(result['latex_results']):
        print(f"❌ 题号超出范围 (1-{len(result['latex_results'])})")
        return
    
    latex_result = result['latex_results'][question_num - 1]
    
    print("="*70)
    print(f"📝 Question {question_num}")
    print("="*70)
    print(latex_result['question_latex'])
    print()
    
    # 显示该题的图片
    question_images = [img for img in images_info if img['question'] == question_num]
    
    if question_images:
        print(f"📷 该题的图片 ({len(question_images)} 张):")
        for img in question_images:
            print(f"\n{'─'*70}")
            print(f"图片 {img['image_index']} (Page {img['page']})")
            print(f"{'─'*70}")
            display(IPImage(filename=img['path'], width=500))
    else:
        print("ℹ️ 该题没有图片")


# 使用示例：显示所有提取的图片
if images_info:
    show_extracted_images(images_info)
    
    print("\n\n")
    
    # 显示第一道有图片的题目
    first_q_with_img = images_info[0]['question'] if images_info else None
    if first_q_with_img:
        show_question_with_images(result, first_q_with_img, images_info)
